In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np 
import json

In [5]:
url = "https://www.adamchoi.co.uk/teamgoals/detailed"

# Start browser
driver = webdriver.Firefox()
driver.get(url)
wait = WebDriverWait(driver, 10)

# Wait for the dropdown to be ready
select_country = wait.until(EC.presence_of_element_located((By.ID, "country")))
select_league = wait.until(EC.presence_of_element_located((By.ID, "league")))
select_c = Select(select_country)
select_l = Select(select_league)

data = []

for option in select_c.options:
    country = option.text.strip()
    print(f"Selecting country: {country}")

    select_c.select_by_visible_text(country)
    
    time.sleep(2)
    
    for league_option in select_l.options:
        league = league_option.text.strip()
        print(f"         Selecting league: {league}")

        select_l.select_by_visible_text(league)

        
        time.sleep(2)
        # Re-parse the page
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        trs = soup.select("table > tbody > tr.ng-scope")
    
        for tr in trs:
            date_td = tr.select_one("td.date-col")
            if not date_td:
                continue
    
            date = date_td.text.strip()
            home_team = tr.select("td.ng-binding")[1].text.strip()
            score = tr.select("td")[3].text.strip()
            away_team = tr.select("td.ng-binding")[2].text.strip()
    
            item = {
                "country": country,
                "date": date,
                "home_team": home_team,
                "score": score,
                "away_team": away_team,
                "League":league
            }
            data.append(item)
driver.quit()

Selecting country: England
         Selecting league: Premier League
         Selecting league: Championship
         Selecting league: League One
         Selecting league: League Two
         Selecting league: National League
         Selecting league: Women's Super League
Selecting country: Spain
         Selecting league: La Liga
         Selecting league: Segunda Division
         Selecting league: Primera Division Women
Selecting country: Germany
         Selecting league: Bundesliga
         Selecting league: Bundesliga 2
         Selecting league: 3. Liga
Selecting country: Italy
         Selecting league: Serie A
         Selecting league: Serie B
Selecting country: France
         Selecting league: Ligue 1
         Selecting league: Ligue 2
Selecting country: Scotland
         Selecting league: SPL
         Selecting league: Scottish Championship
         Selecting league: Scottish League 1
         Selecting league: Scottish League 2
Selecting country: Argentina
         Sel

In [6]:
print(f"✅ Scraped {len(data)} matches in total.")

✅ Scraped 13757 matches in total.


In [7]:
df = pd.DataFrame(data)
df.to_csv("team_goals_2025-2026-all-leagues.csv", index=True)

In [8]:
with open("team_goals_2025-2026-all-leagues.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)